[DNN 기반 회귀 모델 구현]
- 데이터셋: iris.csv
- feature: feature: 3개 (Sepal_Length, Sepal_Width, Petal_Length)
- target: 1개 (Petal_Width)
- 학습: 지도 학습 중 회귀
- 학습 알고리즘: 인공신경망(ANN) 중 심층신경망(DNN)  <- 은닉층이 많은 구성
- 프레임워크: Pytorch
- - -
- 모니터링
    - 기준 설정: 검증 데이터셋의 loss와 score
    - 평가: 학습 데이터셋의 loss 또는 score와 비교 -> 학습 중단 여부 결정
    - 선택: 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장
- - - 
- 진행 파라미터 저장


[1] 모듈 로딩 및 데이터 준비

In [56]:
#모듈 로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [57]:
#활용 패키지 버전 체크
print(f'torch  v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

torch  v.2.4.1
pandas v.2.0.3


In [58]:
#데이터 로딩
data_file='../data/iris.csv'

#csv -> DataFrame
iris_df=pd.read_csv(data_file, usecols=[0,1,2,3])
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


- - - 
[2] 모델 클래스 설계 및 정의
- 클래스 목적: iris 데이터 학습 및 추론
- 클래스 이름: iris_reg_model
- 부모 클래스: nn.Module
- 매개변수: 각 층별 입출력 개수 고정 => 필요 X
- 클래스 속성: feature_df, target_df, n_rows, n_features
- 클래스 기능: __init__() <- 모델 구조 설정, forward() <- 순방향 학습(오버라이딩(상속 관계에서만 가능) 필요)
- 클래스 구조
    - 입력층:  3개 입력 (feature 개수) 10개 출력 (=퍼셉트론 10개) 
    - 은닉층: 10개 입력                30개 출력 (=퍼셉트론 30개)
    - 출력층: 30개 입력                 1개 출력 (target 1개)

- 활성화함수
    - 클래스 형태 (대문자로 시작)
        - nn.MSELoss, nn.ReLU 등
        - _ _init_ _() 메서드에서 사용됨
        - 하나의 층처럼 사용 가능
    - 함수 형태 (소문자로 시작)
        - torch.nn.functional.relu 등
        - forward 메서드에 사용됨

In [59]:
class iris_reg_model(nn.Module):

    #모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(3,10)
        self.hidden_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,1)

    #순방향 학습 진행 메서드
    def forward(self,input_data):

        #입력층
        y=self.in_layer(input_data)     
        y=F.relu(y)                     # y 값: 0 이상

        #은닉층
        y=self.hidden_layer(y)
        y=F.relu(y)

        #출력층_회귀니까 바로 반환
        return self.out_layer(y)


In [60]:
#모델 인스턴스 생성_확인용용
model=iris_reg_model()

print(model)

iris_reg_model(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [61]:
#모델이 사용하는 메모리 정보 확인
summary(model, input_size=(10,3))   #input_size=(n,m) : n행 m열 데이터를 넣었을 때 사용하는 메모리값 보여줌

Layer (type:depth-idx)                   Output Shape              Param #
iris_reg_model                           [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

- - -
[3] 데이터셋 클래스 설계 및 정의
- 데이터셋: iris.csv
- feature: 3개
- target: 1개
- 클래스 이름: iris_ds
- 부모 클래스: utils.data.Dataset
- 클래스 속성(필드): feature_df,target_df,n_rows,n_features
- 필수 메서드: 
    - _ _ init _ _ (self): 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - _ _ len _ _ (self): 데이터 개수 반환
    - _ _ getItem _ _ (self): 특정 인덱스의 feature/target 반환

In [62]:
class iris_dataset(Dataset):
    def __init__(self,feature_df,target_df):
        self.feature_df=feature_df
        self.target_df=target_df

        self.n_rows=feature_df.shape[0]
        self.n_features=feature_df.shape[1]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):

        #tensor화
        feature_ts=torch.FloatTensor(self.feature_df.iloc[index].values)
        target_ts=torch.FloatTensor(self.target_df.iloc[index].values)

        #feature/target 반환
        return feature_ts,target_ts

In [63]:
#데이터셋 인스턴스 생성

feature_df=iris_df[iris_df.columns[:-1]]    #2D (150,3)
target_df=iris_df[iris_df.columns[-1:]]     #2D (150,1)

iris_ds=iris_dataset(feature_df,target_df)

- - -
[4] 학습 준비
- 학습 횟수: EPOCH          #회독 횟수
- 배치 크기: BATCH_SIZE     #한번에 학습할 데이터 양
- 위치 지정: DEVICE         #텐서 저장 및 실행 위치 (GPU or CPU)
- 학습률(LR)
    - 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정
    - 0.001 ~ 0.1 사이 값 주로 사용

In [64]:
#학습 진행 관련 설정

EPOCH=1000         #처음에 1로 설정해서 잘 돌아가는지 확인하고 올리기
BATCH_SIZE=10
BATCH_CNT=iris_df.shape[0]//BATCH_SIZE
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

print(f'BATCH_CNT: {BATCH_CNT}')

BATCH_CNT: 15


- 인스턴스: 모델, 데이터셋, 최적화, (손실함수, 성능지표)

In [65]:
#모델 인스턴스
model=iris_reg_model()

#데이터셋 인스턴스
x_train,x_test,y_train,y_test=train_test_split(feature_df,target_df,random_state=1)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=1)
print(f'x_train: {x_train.shape}, x_test: {x_test.shape}, x_val: {x_val.shape}')
print(f'y_train: {y_train.shape}, y_test: {y_test.shape}, y_val: {y_val.shape}')
print(f'{type(x_train)}, {type(x_test)}, {type(x_val)}')

#iris_ds=iris_dataset(feature_df,target_df)

train_ds=iris_dataset(x_train,y_train)
val_ds=iris_dataset(x_val,y_val)
test_ds=iris_dataset(x_test,y_test)

#최적화 인스턴스
iris_dl=DataLoader(train_ds,batch_size=BATCH_SIZE)

x_train: (84, 3), x_test: (38, 3), x_val: (28, 3)
y_train: (84, 1), y_test: (38, 1), y_val: (28, 1)
<class 'pandas.core.frame.DataFrame'>, <class 'pandas.core.frame.DataFrame'>, <class 'pandas.core.frame.DataFrame'>


In [66]:
#데이터로더, 데이터셋 테스트
for feature,target in iris_dl:
    print(feature.shape,target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [67]:
#최적화 인스턴스: model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

#손실함수 인스턴스: 회귀 => MSE, MAE, RMSE 등 사용
reg_loss=nn.MSELoss()

- - -
[5] 학습 진행

- 모델 저장 준비

In [68]:
#models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

save_path='../models/iris/'

save_File='model.train_wbs.pth'

#모델 구조 및 파라미터 모두 저장
save_model='model_all.pth'

In [69]:
#경로 상 폴더 존재 여부 체크
if not os.path.exists(save_path):
    os.makedirs(save_path)  #하위 폴더까지 생성 가능

- 학습 진행

In [70]:
#학습 확인 w. 손실값, 성능평가 지표

loss_history=[[],[]]
score_history=[[],[]]

BATCH_CNT=iris_ds.n_rows/BATCH_SIZE
print('BATCH_CNT',BATCH_CNT)

#학습 모니터링/스케줄링 설정: loss_history, score_history 활용
# - 임계 기준: 10번
break_count=0
limit=10

for epoch in range(EPOCH):

    #학습 모드로 모델 설정
    model.train()

    total_loss=0
    total_score=0

    for feature_ts,target_ts in iris_dl:

        #학습 진행
        pre_y=model(feature_ts)

        #손실 계산
        loss=reg_loss(pre_y,target_ts)
        total_loss+=loss.item()

        #성능 평가
        score=R2Score()(pre_y,target_ts)
        total_score+=score.item()

        #최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #각 에포크 당 검증 수행: 모델을 검증 모드로 설정
    model.eval()

    with torch.no_grad():
        #검증 데이터셋
        val_feature_ts=torch.FloatTensor(val_ds.feature_df.values)
        val_target_ts=torch.FloatTensor(val_ds.target_df.values)
        
        #평가
        pre_val=model(val_feature_ts)

        #손실 계산
        val_loss=reg_loss(pre_val,val_target_ts)

        #성능 평가
        val_score=R2Score()(pre_val,val_target_ts)

    #손실값, 성능평가값 저장
    loss_history[0].append(total_loss/BATCH_CNT)
    score_history[0].append(total_score/BATCH_CNT)

    loss_history[1].append(val_loss)
    score_history[1].append(val_score)

    #학습 진행 모니터링(스케줄링)_검증 ds 기준 (loss/score 둘 중 하나만 하면 됨)

    #loss
    if len(loss_history[1]) >= 2:
        if loss_history[1][-1] >= loss_history[1][-2]: break_count+=1

    # #score
    # if len(score_history[1])>=2:
    #     if score_history[1][-1] <= score_history[1][-2]: break_count+=1

    #성능이 좋은 학습 가중치 저장
    #save_File=f'model.train_wbs_{epoch}_{val_score}.pth'
    if len(score_history[1])==1:
        #첫번째는 무조건 모델 저장_파라미터만 저장
        torch.save(model.state_dict(),save_path+save_File)
        #모델 전체 저장
        torch.save(model,save_path+save_model)
    else: 
        #두번째부터는 이전보다 성능이 좋으면 저장
        if score_history[1][-1] > max(score_history[1][:-1]):
            torch.save(model.state_dict(),save_path+save_File)
            torch.save(model,save_path+save_model)

    #학습 중단 여부 설정
    if break_count>limit:
        print(f'성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단')
        break

BATCH_CNT 15.0
성능 및 손실 개선이 없어서 109 EPOCH에 학습 중단


In [71]:
val_feature_ts=torch.FloatTensor(val_ds.feature_df.values)
val_target_ts=torch.FloatTensor(val_ds.target_df.values)

val_feature_ts.shape, val_target_ts.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [72]:
print('CNT:',BATCH_CNT)
print(f'loss_history: {loss_history}')
print(f'score_history: {score_history}')

CNT: 15.0
loss_history: [[1.7003047307332357, 1.3944477160771689, 1.160631275177002, 0.9726232409477233, 0.8101300001144409, 0.6716089288393656, 0.5454792698224386, 0.42587889432907106, 0.3191917379697164, 0.22766635417938233, 0.15352982580661773, 0.09911487499872844, 0.06406343256433805, 0.045225932945807776, 0.036119034824272, 0.0325445178275307, 0.03160821044196685, 0.03127817825103799, 0.03097135489806533, 0.030678201032181582, 0.030412379590173563, 0.030167784355580807, 0.029948026376465955, 0.02969929073005915, 0.02947966754436493, 0.02924077392866214, 0.029019915622969468, 0.028801937277118366, 0.028574994951486587, 0.02834744645903508, 0.028117040979365508, 0.027904137161870797, 0.02769077109793822, 0.027471101035674413, 0.027251680319507917, 0.027033415685097376, 0.02681330175449451, 0.02659796935816606, 0.026385424348215263, 0.026167281220356622, 0.02595138605684042, 0.025741144176572563, 0.025525481005509696, 0.02531884318838517, 0.025102319909880558, 0.02488760029276212, 0.

- - - 
모델 저장 방법
- 방법 1: 모델 파라미터만 저장
- 방법 2: 모델 설계 구조 및 파라미터까지 모두 저장

In [73]:
#학습된 모델 파라미터 값 확인
model.state_dict

<bound method Module.state_dict of iris_reg_model(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)>

[방법 1] 모델 파라미터(층별 가중치, 절편)만 저장

In [74]:
#가중치, 절편 로딩: 객체로 로딩 -> 모델의 state_dict 속성에 저장

#객체로 로딩
wb_ts=torch.load(save_path+save_File)
print(type(wb_ts))

#저장
model2=iris_reg_model()     #층마다 가중치, 절편 초기화됨
model2.load_state_dict(wb_ts)

<class 'collections.OrderedDict'>


C:\Users\desktop\AppData\Local\Temp\ipykernel_14140\317353833.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wb_ts=torch.load(save_path+save_File)


<All keys matched successfully>

In [75]:
print(model2)

iris_reg_model(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
